---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

def blight_model():
    
    #Reading datasets
    train = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test = pd.read_csv('test.csv')
    
    address = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')

    
    #Merging lat and lon in data and train sets
    addressMerge = address.set_index('address').join(latlons.set_index('address'))
    train = train.set_index('ticket_id').join(addressMerge.set_index('ticket_id'))
    test = test.set_index('ticket_id').join(addressMerge.set_index('ticket_id'))
    
    #Cleaning data
    train = train[(train['compliance'] == 0) | (train['compliance'] == 1)] #Removing not responsible people
    
    #Features to remove
    features_to_drop_test = ['fine_amount', 'violator_name', 'zip_code', 'country', 'city',
                          'inspector_name', 'violation_street_number', 'violation_street_name',
                          'violation_zip_code', 'violation_description',
                          'mailing_address_str_number', 'mailing_address_str_name',
                          'non_us_str_code', 'grafitti_status', 'violation_code']
    
    features_to_drop_train = [
    'balance_due',
    'collection_status',
    'compliance_detail',
    'payment_amount',
    'payment_date',
    'payment_status'] + features_to_drop_test
    
    train.drop(features_to_drop_train, axis = 1, inplace = True) #removing not important features in train set
    train = train[train.hearing_date.isnull() == False] #removing nan values in 'hearing_date' in train set
    
    test.drop(features_to_drop_test, axis = 1, inplace = True) #removing not important features in test set
    test.hearing_date = test.hearing_date.fillna(method = 'bfill') #filling nan values in 'hearing_date' in test set
    
    
    #Feature Engineering: number of days between hearing and ticket issued date
    train['gapDays'] = train.apply(lambda x: (datetime.strptime(x['hearing_date'], '%Y-%m-%d %H:%M:%S') - 
                                   datetime.strptime(x['ticket_issued_date'], '%Y-%m-%d %H:%M:%S')).days, axis = 1)
    
    train.drop(['hearing_date', 'ticket_issued_date'], axis = 1, inplace = True) #Removing hearing and ticket issued date
    
    test['gapDays'] = test.apply(lambda x: (datetime.strptime(x['hearing_date'], '%Y-%m-%d %H:%M:%S') - 
                                   datetime.strptime(x['ticket_issued_date'], '%Y-%m-%d %H:%M:%S')).days, axis = 1)
    
    test.drop(['hearing_date', 'ticket_issued_date'], axis = 1, inplace = True) #Removing hearing and ticket issued date
    
    #Feature Engineering: filling nans
    train.state = train.state.fillna(method = 'bfill')
    train.lat = train.lat.fillna(method = 'bfill')
    train.lon = train.lon.fillna(method = 'bfill')
    
    test.state = test.state.fillna(method = 'bfill')
    test.lat = test.lat.fillna(method = 'bfill')
    test.lon = test.lon.fillna(method = 'bfill')
    
    #Preprocessing: LabelEncoder for str columns
    le = preprocessing.LabelEncoder()
    
        #agency_name
    le.fit(train.agency_name)
    train['agency_name'] = le.transform(train.agency_name)
    
    le.fit(test.agency_name)
    test['agency_name'] = le.transform(test.agency_name)
        
        #state
    le.fit(train.state)
    train['state'] = le.transform(train.state)
    
    le.fit(test.state)
    test['state'] = le.transform(test.state)
    
        #disposition
    le.fit(train.disposition)
    train['disposition'] = le.transform(train.disposition)
    
    le.fit(test.disposition)
    test['disposition'] = le.transform(test.disposition)
    
    
    #Model: Random Forest
    y_train = train['compliance']
    X_train = train.ix[:,train.columns != 'compliance']
    
    X_test = test
    
    rf = RandomForestClassifier(max_features = 8, random_state = 0)
    rf.fit(X_train, y_train)
    
    y_test = rf.predict(X_test)
    y_test_prob = rf.predict_proba(X_test)[:,1]
    
    #print('Accuracy of RF classifier on training set: {:.2f}'
    # .format(rf.score(X_train, y_train)))
    #print('Accuracy of RF classifier on test set: {:.2f}'
    # .format(rf.score(X_test, y_test)))
    
    #Probability
    test_prob = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    test_prob['compliance'] = y_test_prob
    
    return test_prob.set_index('ticket_id').compliance

blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.6
285362    0.2
285361    0.4
285338    0.2
285346    0.2
285345    0.2
285347    0.4
285342    1.0
285530    0.1
284989    0.4
285344    0.8
285343    0.1
285340    0.3
285341    0.5
285349    0.3
285348    0.3
284991    0.4
285532    0.4
285406    0.2
285001    0.4
285006    0.4
285405    0.6
285337    0.2
285496    0.5
285497    0.5
285378    0.4
285589    0.0
285585    0.3
285501    0.7
285581    0.1
         ... 
376367    0.0
376366    0.3
376362    0.0
376363    0.6
376365    0.0
376364    0.3
376228    0.7
376265    0.0
376286    0.8
376320    0.3
376314    0.3
376327    1.0
376385    1.0
376435    1.0
376370    1.0
376434    0.7
376459    0.7
376478    0.0
376473    0.0
376484    0.5
376482    0.2
376480    0.2
376479    0.2
376481    0.2
376483    0.2
376496    0.2
376497    0.2
376499    0.2
376500    0.2
369851    1.0
Name: compliance, dtype: float64